In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, desc, lower
import re
EXPRESSIONS_SIZE = 3

from utils import get_session

spark = get_session()

df = spark.read.csv('hdfs:///tweets/debate-tweets-001.tsv', sep='\t').select('_c1', '_c7', '_c8')
df.show()

Setting default log level to "

22/12/08 22:47:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+--------------------+--------------------+----------+
|                 _c1|                 _c7|       _c8|
+--------------------+--------------------+----------+
|@anacddd verdade,...|Wed Oct 15 14:31:...|2014-10-15|
|              Que ñ*|Wed Oct 15 14:31:...|2014-10-15|
| Vou quebrar a Bruna|Wed Oct 15 14:31:...|2014-10-15|
|agora vou p segun...|Wed Oct 15 14:31:...|2014-10-15|
|Me sinto tão bem ...|Wed Oct 15 14:31:...|2014-10-15|
|Eu estou aqui, de...|Wed Oct 15 14:31:...|2014-10-15|
|Quando vai embora...|Wed Oct 15 14:31:...|2014-10-15|
|@paynecaralhudo k...|Wed Oct 15 14:31:...|2014-10-15|
|Conceição da Barr...|Wed Oct 15 14:31:...|2014-10-15|
| @Maniavato te amo ♥|Wed Oct 15 14:31:...|2014-10-15|
|Alg me curtindo rs ♡|Wed Oct 15 14:31:...|2014-10-15|
|@MiiluAA No, porq...|Wed Oct 15 14:31:...|2014-10-15|
|#EMABiggestFansJu...|Wed Oct 15

In [2]:
#Filter by tweets that contains 'Aécio' and match with other tweets that have the same sentences
from pyspark.ml.feature import NGram

df2 = df.select(split(lower(df._c1),' ').alias('words')).na.drop()
ngram_gen = NGram(n=EXPRESSIONS_SIZE, inputCol="words", outputCol="phrases")
processed_df = ngram_gen.transform(df2)

df3 = processed_df.select(explode(processed_df.phrases)).withColumnRenamed('col','sentences')
df4 = df3.filter(df3.sentences.contains('Aecio') | df3.sentences.contains('aecio') | df3.sentences.contains('aécio') | df3.sentences.contains('Aécio'))
df5 = df4.groupBy(df4.sentences).count().orderBy(desc('count'))
df5.show()

+--------------+-----+
|     sentences|count|
+--------------+-----+
|   que o aécio|  410|
| dilma e aécio|  241|
|votar no aécio|  203|
|    se o aécio|  189|
|     o aécio é|  149|
|   que o aecio|  135|
|    do aécio é|  125|
|     e o aécio|  122|
| aécio e dilma|  114|
|   o aécio não|  106|
|     aécio é o|  104|
| vota no aécio|   95|
|   o aécio vai|   90|
|  esse aécio é|   88|
|    do aécio e|   83|
|votar no aecio|   81|
|    aécio é um|   81|
|o aécio ganhar|   79|
|     o aécio e|   79|
|de aécio neves|   78|
+--------------+-----+
only showing top 20 rows



In [3]:
spark.stop()